# Tutorial Convert Solvency 2 XBRL-instances to CSV, HTML and pickles

This tutorial describes how to convert XBRL-instances to csv-, html- and pickle files per template.

We use Arelle, a open source package for processing XBRL. In addition this repository contains code to process the Solvency 2 and FTK instances efficiently.

In [1]:
from arelle import ModelManager, Cntlr, ModelXbrl, XbrlConst, RenderingEvaluator, \
                   ViewFileRenderedGrid, ModelFormulaObject

In [2]:
import src
import pandas as pd
from os import listdir, walk, makedirs, environ
from os.path import isfile, join, exists, basename
from datetime import datetime

## Initialize the Arelle model manager

First we specify the directories with the taxonomy and instances. You can put your own instances in the data/instances directory, or you can specify here the directories that you want to use.

In [3]:
# the taxonomy should be data/taxonomy/arelle
# the instances you want to use should be in data/instances

XBRL_TAXONOMY_PATH = join('..', 'data', 'taxonomy')
XBRL_INSTANCES_PATH = join('..', 'data', 'instances')

LANGUAGE = "en-GB"
environ['XDG_CONFIG_HOME'] = XBRL_TAXONOMY_PATH

# The role defined in the model.xsd schema for resources representing codes of rows or columns is
euRCcode = 'http://www.eurofiling.info/xbrl/role/rc-code'

To process XBRL, we need a controller and a modelmanager object. 

In the controller you can specify logging. Here we have set logging to print in this notebook.

In [4]:
# Now we make a modelmanager
# logFileName = "logToPrint" -> logging is print to notebook
# logFileName = "arelle.log" -> logging is to filename (use .json or .xml for specific format)

controller = Cntlr.Cntlr(logFileName = "logToPrint")
controller.webCache.workOffline = True
controller.logger.messageCodeFilter = None

modelmanager = ModelManager.initialize(controller)
modelmanager.defaultLang = LANGUAGE
modelmanager.formulaOptions = ModelFormulaObject.FormulaOptions()
modelmanager.loadCustomTransforms()

## Read XBRL-instance in the modelmanager

Now we are able to read and process an XBRL-instance.

We read the example instances provided with the taxonomy.

In [5]:
# the example instance of the quarterly templates for solo
instance_name = 'qrs_240_instance.xbrl'  

# the example instance of the annual templates
# instance_name = 'aeb_240_instance.xbrl'

# the example instance of the FTK assets templates
# instance_name = 'DNB-NR_FTK-2019-06_2019-12-31_MOD_FTK-BEL.XBRL'

In [6]:
xbrl_instance = ModelXbrl.load(modelmanager, join(XBRL_INSTANCES_PATH, instance_name))
RenderingEvaluator.init(xbrl_instance)

2020-09-02 11:47:44,470 [] Formula xpath2 grammar initialized in 0,89 secs - 

2020-09-02 11:47:45,695 [info:profileActivity] ... formula parameter checks 1,217 secs
 - qrs_240_instance.xbrl 

2020-09-02 11:47:47,118 [info:profileActivity] ... custom function checks and compilation 1,423 secs
 - qrs_240_instance.xbrl 



## Convert XBRL-instance to CSV and Pandas-pickle

For each template or table in the instance we export the results to a csv file and a Pandas pickle-file. 

A Pandas pickle-file maintains the correct indices, whereas the csv does not, so if you want to access the data read the pickle (we included an example below).

The csv-files and the pickle-files are stored in a subdirectory identical to the name of the XBRL-instance (without extension)

In [7]:
# The location of the csv-files
subdir = basename(instance_name).split(".")[0]

In [8]:
# get tables in instance and sort by short name and print the first ten tables
tables = list(xbrl_instance.modelRenderingTables)
tables.sort(key = lambda table: table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode))
for table in tables[0:10]:
    print(table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode))

S.01.01.02.01
S.01.02.01.01
S.02.01.02.01
S.05.01.02.01
S.05.01.02.02
S.06.02.01.01
S.06.02.01.02
S.06.03.01.01
S.08.01.01.01
S.08.01.01.02


In [9]:
# create csv and pickle files
# time_stamp = datetime.now().strftime("%Y_%m_%d-%H_%M_%S")

# use verbose_labels = False if you want the row-column code as column names
# use verbose_labels = True if you want labels as column names

for table in tables:
    obj = src.generateCSV.generateCSVTables(xbrl_instance, join(XBRL_INSTANCES_PATH, subdir), 
                                            table = table, 
                                            lang = LANGUAGE,
                                            verbose_labels = False)

2020-09-02 11:47:48,503 [xbrlte:closedDefinitionNodeZeroCardinality] Closed definition node s2md_c74 does not contribute at least one structural node - http://eiopa.europa.eu/eu/xbrl/s2md/fws/solvency/solvency2/2019-07-15/tab/s.01.01.02.01/s.01.01.02.01-rend.xml 12, 16

2020-09-02 11:47:48,908 []  ... saved output ..\data\instances\qrs_240_instance\S.01.01.02.01.csv and .pickle - 

2020-09-02 11:47:49,806 [xbrlte:closedDefinitionNodeZeroCardinality] Closed definition node s2md_c499 does not contribute at least one structural node - http://eiopa.europa.eu/eu/xbrl/s2md/fws/solvency/solvency2/2019-07-15/tab/s.01.02.01.01/s.01.02.01.01-rend.xml 12, 16

2020-09-02 11:47:49,854 []  ... saved output ..\data\instances\qrs_240_instance\S.01.02.01.01.csv and .pickle - 

2020-09-02 11:47:50,877 []  ... saved output ..\data\instances\qrs_240_instance\S.02.01.02.01.csv and .pickle - 

2020-09-02 11:47:52,070 []  ... saved output ..\data\instances\qrs_240_instance\S.05.01.02.01.csv and .pickle - 

2

In [10]:
# create html files
# for large tables this will take a lot of time!

for table in tables:
    ViewFileRenderedGrid.viewRenderedGrid(xbrl_instance,
             join(XBRL_INSTANCES_PATH, subdir, table.genLabel(lang = LANGUAGE,strip = True, role = euRCcode)+".html"),
             viewTblELR = table,
             lang = LANGUAGE, 
             sourceView = None,
             diffToFile = False)

2020-09-02 11:48:11,908 [xbrlte:closedDefinitionNodeZeroCardinality] Closed definition node s2md_c74 does not contribute at least one structural node - http://eiopa.europa.eu/eu/xbrl/s2md/fws/solvency/solvency2/2019-07-15/tab/s.01.01.02.01/s.01.01.02.01-rend.xml 12, 16

2020-09-02 11:48:11,946 [info] Saved output HTML to ..\data\instances\qrs_240_instance\S.01.01.02.01.html - qrs_240_instance.xbrl 

2020-09-02 11:48:11,946 [xbrlte:closedDefinitionNodeZeroCardinality] Closed definition node s2md_c499 does not contribute at least one structural node - http://eiopa.europa.eu/eu/xbrl/s2md/fws/solvency/solvency2/2019-07-15/tab/s.01.02.01.01/s.01.02.01.01-rend.xml 12, 16

2020-09-02 11:48:11,977 [info] Saved output HTML to ..\data\instances\qrs_240_instance\S.01.02.01.01.html - qrs_240_instance.xbrl 

2020-09-02 11:48:12,046 [info] Saved output HTML to ..\data\instances\qrs_240_instance\S.02.01.02.01.html - qrs_240_instance.xbrl 

2020-09-02 11:48:12,425 [info] Saved output HTML to ..\data\i

## Example to read a template from the pickle files

The easiest way to access the data of a separate template is to read the corresponding pickle-file.

In [11]:
t = tables[10].genLabel(lang = LANGUAGE,strip = True, role = euRCcode)
df = pd.read_pickle(join(XBRL_INSTANCES_PATH, subdir, t + ".pickle"))
df

,,,,,,"S.08.02.01.01,C0060","S.08.02.01.01,C0080","S.08.02.01.01,C0110","S.08.02.01.01,C0120","S.08.02.01.01,C0130","S.08.02.01.01,C0140","S.08.02.01.01,C0150","S.08.02.01.01,C0160","S.08.02.01.01,C0170","S.08.02.01.01,C0180","S.08.02.01.01,C0190","S.08.02.01.01,C0200","S.08.02.01.01,C0210","S.08.02.01.01,C0220","S.08.02.01.01,C0230"
entity,period,"S.08.02.01.01,C0440","S.08.02.01.01,C0040","S.08.02.01.01,C0070","S.08.02.01.01,C0090",,,,,,,,,,,,,,,
0LFF1WMNTWG5PTIYYI38,2019-12-31,1,ISIN/RB3299788372,1,ISIN/HV1587599059,Ring Fenced Funds,Unit-linked or index-linked,Macro hedge [MA],8.654659e+08,5 - FL-FX: Deliver floating-for-fixed,96272553.07,5.749187e+08,3.812666e+08,308859.0,818385.0,2.704652e+08,64831970.44,128634715.4,2015-07-08 00:00:00,1.789155e+08


## Validate instance from Arelle

It should be possible to validate the instance (performing the validation rules within the taxonomy) with Arelle with the following code. But we did not test this!

In [12]:
# from arelle import Validate
# Validate.validate(xbrl_instance)